# Using RDF-Star and SPARQL-Star with the AllegroGraph Python Interface

This notebook will show the basic functionality of **RDF-star** and **SPARQL-star** using the `agraph-python` interface. If you want to read more about **RDF-star** and **SPARQL-star** you can visit the [W3C specification document](https://www.w3.org/2021/12/rdf-star.html#overview).

If you are connecting in a regular jupyter notebook and have activated our `ag-tutorial` environment then you do not need to run the following cell to install agraph-python. However if you are in a Google Colab please run the following cell.

In [ ]:
%pip install agraph-python pycurl

We will only set the environment variables for AGRAPH_PASSWORD, AGRAPH_HOST and AGRAPH_PORT. If connecting to a cloud server you will not need to change the port, but please copy the host from your server. We have left a (nonworking) example in place.

In [ ]:
import os
os.environ['AGRAPH_USER'] = 'admin'
os.environ['AGRAPH_PASSWORD'] = 'GrMEKDvQFaN2bkrHJeiCbv'
os.environ['AGRAPH_HOST'] = 'https://ag197y8xsj2epl2e.allegrograph.cloud'
os.environ['AGRAPH_PORT'] = '443'

In [68]:
from franz.openrdf.connect import ag_connect
from franz.openrdf.query.query import QueryLanguage

conn = ag_connect('rdf-star-example')
conn.setNamespace(':', 'http://franz.com/')

To use the **RDF-star** and **SPARQL-star** capabilities the user must always enable those features by running the `enableRDFStar()` method on the connection object. This only has to be done once. Once the repository has been created and every future time the user connects the capabilities will still be enabled.

In [69]:
conn.enableRDFStar()

We will start by adding some triples using the `Turtle-star` syntax. In this example we add will add `:Bilbo :hasHeir :Frodo` both as an **asserted triple** and as a **quoted triple**. With this example, because the aforementioned triple is both **asserted** and **quoted** we can query for the both, as will be shown later, but for now we are only adding the data to the graph.

In [70]:
conn.addData("""
        PREFIX : <http://franz.com/>
        
        :Frodo :name "Frodo Baggins" ; a :Hobbit .
        :Bilbo :name "Bilbo Baggins" ; a :Hobbit.
        :Lobelia :name "Lobelia Sackville-Baggins" ; a :Hobbit .
        
        :Bilbo :hasHeir :Frodo . #asserted triple
        << :Bilbo :hasHeir :Frodo >> :dislikedBy :Lobelia . #quoted triple """)

In [71]:
conn.size()

9

We can now also view this **quoted** triple in Gruff. Notice how the `:dislikedBy` predicate points from the center of the `:heirOf` to `:Lobelia`

![heirOf](images/heirOf.png)

We can query for elements of **quoted** and **asserted** triples using the usual python querying methods.

In [72]:
query_string = """
    PREFIX : <http://franz.com/>
    
    SELECT ?being1 ?being2 ?bitterOldPerson WHERE {
        << ?being1 :hasHeir ?being2 >> :dislikedBy ?bitterOldPerson . }"""
result = conn.executeTupleQuery(query_string).toPandas()
result

,being1,being2,bitterOldPerson
0,<http://franz.com/Bilbo>,<http://franz.com/Frodo>,<http://franz.com/Lobelia>


We can also query use the following syntax, called the [annotation syntax](https://www.w3.org/2021/12/rdf-star.html#dfn-annotation-syntax). This syntax is used to identify triples that are both **asserted** and **quoted**

In [73]:
query_string = """
    PREFIX : <http://franz.com/>
    
    SELECT ?being1 ?being2 ?bitterOldPerson where {
        ?being1 :hasHeir ?being2 {| :dislikedBy ?bitterOldPerson |} }"""
result = conn.executeTupleQuery(query_string).toPandas()
result

,being1,being2,bitterOldPerson
0,<http://franz.com/Bilbo>,<http://franz.com/Frodo>,<http://franz.com/Lobelia>


Because the triple is **asserted** we can also query for the info without including any **RDF-star** syntax.

In [74]:
query_string = """
    PREFIX : <http://franz.com/>
    
    SELECT ?being1 ?being2 WHERE {
        ?being1 :hasHeir ?being2 }"""
result = conn.executeTupleQuery(query_string).toPandas()
result

,being1,being2
0,<http://franz.com/Bilbo>,<http://franz.com/Frodo>


It is very important to be cognizant of the difference between **asserted** and **quoted** triples. The following query *only* quotes a triple, and does *not* assert it.

In [75]:
conn.addData("""
    PREFIX : <http://franz.com/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    
    :Gollum :name "Gollum" ; a :Stoor .
    :Stoor rdfs:subClassOf :Hobbit .
    
    << :Gollum :willNotHarm :Frodo >> :accordingTo :Gollum . """)

When we query for who `:willNotHarm :Frodo` we get no results, because this is only `:accordingTo :Gollum` and is not a fact in the graph.

In [76]:
query_string = """
    PREFIX : <http://franz.com/>
    
    SELECT ?being WHERE {
        ?being :willNotHarm :Frodo . }"""
result = conn.executeTupleQuery(query_string).toPandas()
result

,being


If you use the Annotation Syntax mentioned above the following `Turtle-star` will result in **quoted** and **asserted** triple.

In [77]:
conn.addData("""
    PREFIX : <http://franz.com/>
    
    :TheRing :name "Ring of Power" ; a :RingOfPower .
    
    :Gollum :loves :TheRing {| :accordingTo :Gollum |} . """)

Now, if we query who `:loves :TheRing` we get the correct result, because the statement was added as a **quoted** and **asserted** triple.

In [78]:
query_string = """
    PREFIX : <http://franz.com/>
    
    SELECT ?being WHERE {
        ?being :loves :TheRing . }"""
result = conn.executeTupleQuery(query_string).toPandas()
result

,being
0,<http://franz.com/Gollum>


## Adding and querying data using Agraph Python Interface without Turtle or SPARQL

It is also possible to add data to the graph without writing `Turtle-star` and querying the graph without `SPARQL-star`. First we import the `QuotedTriple` class and create a namespace that can be used locally with our python.

In [79]:
from franz.openrdf.model import QuotedTriple

f = conn.namespace('http://franz.com/')

Here we create a **quoted** triple:

In [80]:
quoted_triple = QuotedTriple(f.Frodo, f.carries, f.TheRing)

We can access the elements of the **quoted** triple using the usual methods:

In [81]:
quoted_triple.getSubject()

<http://franz.com/Frodo>

Now we can add a **quoted** triple using the usual `addTriple` method. This is the same as the following turtle syntax:
```
    PREFIX : <http://franz.com/>
    
    << :Frodo :carries :TheRing >> :unbeknownstTo :Sauron .
```
Note that this is only adding a **quoted** triple, and *not* an **asserted** triple.

In [82]:
conn.addTriple(quoted_triple, f.unbeknownstTo, f.Sauron)

Querying using the `getStatements` method we see that the subject is the **quoted** triple.

In [83]:
df = conn.getStatements(None, f.unbeknownstTo, None).toPandas()
df

,s,p,o,g
0,<< <http://franz.com/Frodo> <http://franz.com/...,<http://franz.com/unbeknownstTo>,<http://franz.com/Sauron>,None


Taking a closer look at the **quoted** triple in the pandas DataFrame

In [84]:
df['s'][0]

'<< <http://franz.com/Frodo> <http://franz.com/carries> <http://franz.com/TheRing> >>'

Also, if we iterate through all statements generated by the `getStatements` method, we can see that the value of the `Subject` of the following query is a `QuotedTriple`, and that we can access the individual elements (i.e. `Subject`, `Predicate`, `Object`).

In [85]:
statements = conn.getStatements(None, f.unbeknownstTo, None)
with statements:
    for statement in statements:
        quoted_triple = statement.getSubject() # the quoted triple 
        print(type(quoted_triple))
        print(quoted_triple.getSubject(), quoted_triple.getPredicate(), quoted_triple.getObject())
    

<class 'franz.openrdf.model.value.QuotedTriple'>
<http://franz.com/Frodo> <http://franz.com/carries> <http://franz.com/TheRing>


Finally, can see the full graph here!:
    
![all relationships](images/fullRelationships.png)